In [2]:
# Importing the libraries needed in this project
import os
import pandas as pd
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col, when, lower, isnull, year, month, dayofmonth, hour, weekofyear, dayofweek, date_format, to_date
from datetime import timedelta, datetime

In [3]:
spark = create_spark_session()

In [4]:
immigration = pd.read_csv('immigration_data_sample.csv')
immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [5]:
not_useful = ["count", "entdepa", "entdepd", "matflag", "fltno", "dtaddto", "admnum", "i94bir", "dtadfile", "visapost", "occup", "entdepu", "insnum"]
immigration.drop(not_useful, axis = 1, inplace=True)

In [6]:
immigration.dropna(how='any', subset=['i94addr'])
immigration.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,depdate,i94visa,biryear,gender,airline,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,20573.0,2.0,1955.0,F,JL,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,20568.0,2.0,1990.0,M,*GA,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,20571.0,2.0,1940.0,M,LH,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,20581.0,2.0,1991.0,M,QR,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,20553.0,2.0,1997.0,F,NaN,WT


In [7]:
immigration['i94yr'] = immigration['i94yr'].astype(int)
immigration['i94mon'] = immigration['i94mon'].astype(int)
immigration['cicid'] = immigration['cicid'].astype(int)

In [12]:
immigration = immigration.rename(columns={'cicid':'ID', 'i94yr':'year', 'i94mon':'month', 'i94cit':'native_country', 'i94res':'residence_country', 'i94port':'port of admission', 'arrdate':'arrival_date', 'i94mode':'mode_of_trans', 'i94addr':'arrival_state', 'depdate':'departure_date', 'i94visa':'visa_code', 'biyear':'birth_year', 'dtaddto':'due_date'})
immigration.head()

,Unnamed: 0,ID,year,month,native_country,residence_country,port of admission,arrival_date,mode_of_trans,arrival_state,departure_date,visa_code,biryear,gender,airline,visatype
0,2027561,4084316,2016,4,209.0,209.0,HHW,20566.0,air,HI,20573.0,Pleasure,1955.0,F,JL,WT
1,2171295,4422636,2016,4,582.0,582.0,MCA,20567.0,air,TX,20568.0,Pleasure,1990.0,M,*GA,B2
2,589494,1195600,2016,4,148.0,112.0,OGG,20551.0,air,FL,20571.0,Pleasure,1940.0,M,LH,WT
3,2631158,5291768,2016,4,297.0,297.0,LOS,20572.0,air,CA,20581.0,Pleasure,1991.0,M,QR,B2
4,3032257,985523,2016,4,111.0,111.0,CHM,20550.0,land,NY,20553.0,Pleasure,1997.0,F,NaN,WT


In [13]:
immigration.loc[(immigration.mode_of_trans == 1.0), 'mode_of_trans'] = 'air'
immigration.loc[(immigration.mode_of_trans == 2.0), 'mode_of_trans'] = 'see'
immigration.loc[(immigration.mode_of_trans == 3.0), 'mode_of_trans'] = 'land'
immigration.loc[(immigration.mode_of_trans == 9.0), 'mode_of_trans'] = 'not reported'

In [14]:
immigration.head()

,Unnamed: 0,ID,year,month,native_country,residence_country,port of admission,arrival_date,mode_of_trans,arrival_state,departure_date,visa_code,biryear,gender,airline,visatype
0,2027561,4084316,2016,4,209.0,209.0,HHW,20566.0,air,HI,20573.0,Pleasure,1955.0,F,JL,WT
1,2171295,4422636,2016,4,582.0,582.0,MCA,20567.0,air,TX,20568.0,Pleasure,1990.0,M,*GA,B2
2,589494,1195600,2016,4,148.0,112.0,OGG,20551.0,air,FL,20571.0,Pleasure,1940.0,M,LH,WT
3,2631158,5291768,2016,4,297.0,297.0,LOS,20572.0,air,CA,20581.0,Pleasure,1991.0,M,QR,B2
4,3032257,985523,2016,4,111.0,111.0,CHM,20550.0,land,NY,20553.0,Pleasure,1997.0,F,NaN,WT


In [15]:
immigration.loc[(immigration.visa_code == 1.0), 'visa_code'] = 'Business'
immigration.loc[(immigration.visa_code == 2.0), 'visa_code'] = 'Pleasure'
immigration.loc[(immigration.visa_code == 3.0), 'visa_code'] = 'Student'

In [16]:
immigration.head()

,Unnamed: 0,ID,year,month,native_country,residence_country,port of admission,arrival_date,mode_of_trans,arrival_state,departure_date,visa_code,biryear,gender,airline,visatype
0,2027561,4084316,2016,4,209.0,209.0,HHW,20566.0,air,HI,20573.0,Pleasure,1955.0,F,JL,WT
1,2171295,4422636,2016,4,582.0,582.0,MCA,20567.0,air,TX,20568.0,Pleasure,1990.0,M,*GA,B2
2,589494,1195600,2016,4,148.0,112.0,OGG,20551.0,air,FL,20571.0,Pleasure,1940.0,M,LH,WT
3,2631158,5291768,2016,4,297.0,297.0,LOS,20572.0,air,CA,20581.0,Pleasure,1991.0,M,QR,B2
4,3032257,985523,2016,4,111.0,111.0,CHM,20550.0,land,NY,20553.0,Pleasure,1997.0,F,NaN,WT


In [17]:
immigration=immigration.dropna(how='any', subset=['arrival_state'])
immigration.isna().sum()

Unnamed: 0             0
ID                     0
year                   0
month                  0
native_country         0
residence_country      0
port of admission      0
arrival_date           0
mode_of_trans          0
arrival_state          0
departure_date        42
visa_code              0
biryear                0
gender               127
airline               23
visatype               0
dtype: int64

In [18]:
demographics= pd.read_csv('us-cities-demographics.csv', sep=';')
demographics.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


In [21]:
pd.to_timedelta(immigration['arrival_date'], unit='D') + pd.Timestamp('1960-1-1')
pd.to_timedelta(immigration['departure_date'], unit='D') + pd.Timestamp('1960-1-1')
immigration.head()

,Unnamed: 0,ID,year,month,native_country,residence_country,port of admission,arrival_date,mode_of_trans,arrival_state,departure_date,visa_code,biryear,gender,airline,visatype
0,2027561,4084316,2016,4,209.0,209.0,HHW,20566.0,air,HI,20573.0,Pleasure,1955.0,F,JL,WT
1,2171295,4422636,2016,4,582.0,582.0,MCA,20567.0,air,TX,20568.0,Pleasure,1990.0,M,*GA,B2
2,589494,1195600,2016,4,148.0,112.0,OGG,20551.0,air,FL,20571.0,Pleasure,1940.0,M,LH,WT
3,2631158,5291768,2016,4,297.0,297.0,LOS,20572.0,air,CA,20581.0,Pleasure,1991.0,M,QR,B2
4,3032257,985523,2016,4,111.0,111.0,CHM,20550.0,land,NY,20553.0,Pleasure,1997.0,F,NaN,WT


In [4]:
airports=pd.read_csv('airport-codes_csv.csv')
airports.head()

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"
2,00AK,small_airport,Lowell Field,450.0,NaN,US,US-AK,Anchor Point,00AK,NaN,00AK,"-151.695999146, 59.94919968"
3,00AL,small_airport,Epps Airpark,820.0,NaN,US,US-AL,Harvest,00AL,NaN,00AL,"-86.77030181884766, 34.86479949951172"
4,00AR,closed,Newport Hospital & Clinic Heliport,237.0,NaN,US,US-AR,Newport,NaN,NaN,NaN,"-91.254898, 35.6087"


In [5]:
temp=pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')
temp.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [8]:
temp.isna().sum()

dt                                    0
AverageTemperature               364130
AverageTemperatureUncertainty    364130
City                                  0
Country                               0
Latitude                              0
Longitude                             0
dtype: int64

In [10]:
temp.shape

(8599212, 7)